In [1]:
##pacotes nescessários
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, Dropout, GlobalAveragePooling1D, Bidirectional, GRU

In [2]:
##importando os dados
df_texto = pd.read_csv(r"C:\Users\ander\Documents\projetos data science\projeto genética\textos_tratados.csv", delimiter=';',
                      header = None)

In [3]:
df_dado = pd.read_csv(r"msk-redefining-cancer-treatment\training_variants.zip", header = 0)

In [4]:
df = pd.merge(df_dado, df_texto, how = "inner", left_index=True, right_index=True).fillna("")

In [5]:
df.rename(columns={ 0 : 'Texto'}, inplace = True)

In [6]:
df.head()

,ID,Gene,Variation,Class,Texto
0,0,FAM58A,Truncating Mutations,1,"cyclin,depend,kinas,cdk,regul,varieti,fundamen..."
1,1,CBL,W802*,2,"abstract,background,non,small,cell,lung,cancer..."
2,2,CBL,Q249E,2,"abstract,background,non,small,cell,lung,cancer..."
3,3,CBL,N454D,3,"recent,evid,demonstr,acquir,uniparent,disomi,a..."
4,4,CBL,L399V,4,"oncogen,mutat,monomer,casita,b,lineag,lymphoma..."


In [7]:
max_features = 500 # Número máximo de palavras a serem mantidas no vocabulário
max_seq_len = 1000 # Comprimento máximo das sequências de entrada
embedding_dim = 100 # Tamanho da dimensão de embedding
num_filters = 64 # Número de filtros na camada de convolução
filter_size = 5 # Tamanho do filtro na camada de convolução

In [8]:
# Dividindo o conjunto de dados em treino e teste
X = df[['Texto', "Gene"]]
y = df['Class']

In [9]:
y = pd.get_dummies(y)

In [10]:
X_train_cru, X_test_cru, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
##tokenizando
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts([str(x) for x in X_train_cru['Texto'].tolist()])

In [12]:
X_train_token = tokenizer.texts_to_sequences([str(x) for x in X_train_cru['Texto'].tolist()])
X_test_token = tokenizer.texts_to_sequences([str(x) for x in X_test_cru['Texto'].tolist()])

In [13]:
# Padronizando o tamanho das sequências usando o pad_sequences
X_train_token = pd.DataFrame(pad_sequences(X_train_token, maxlen= 1000))
X_test_token = pd.DataFrame(pad_sequences(X_test_token, maxlen= 1000))

In [14]:
##fazer o encoding dos dados de entrada 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(df["Gene"])

X_train_encoded = pd.DataFrame(le.transform(X_train_cru["Gene"]))

X_test_encoded = pd.DataFrame(le.transform(X_test_cru["Gene"]))

In [15]:
## juntando tokenizado com encodado

X_train_token["Gene"] = X_train_encoded

In [16]:
X_test_token["Gene"] = X_test_encoded

In [17]:
X_train = X_train_token.values
X_test = X_test_token.values

In [18]:
max_features = 1000 # Número máximo de palavras a serem mantidas no vocabulário
max_seq_len = 1000 # Comprimento máximo das sequências de entrada
embedding_dim = 50 # Tamanho da dimensão de embedding
num_filters = 64 # Número de filtros na camada de convolução
filter_size = 5 # Tamanho do filtro na camada de convolução

vocab_size = len(tokenizer.word_index) + 1 # tamanho do vocabulário
lstm_units = 64 # unidades da LSTM

maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [19]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length= (max_seq_len + 1)))
model.add(Conv1D(num_filters, filter_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(9, activation='softmax'))

In [49]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length= (max_seq_len + 1)))
model.add(LSTM(lstm_units))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation="softmax"))

In [69]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length= (max_seq_len + 1)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(9, activation='softmax'))

In [20]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length= (max_seq_len + 1)))
model.add(Bidirectional(LSTM(32, return_sequences=True))) #. input_shape = (None, *x_train.shape) ,
model.add(Dropout(0.20))
model.add(Bidirectional(LSTM(32, return_sequences=False, name="three")))
model.add(Dropout(0.10))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.10))
model.add(Dense(9, activation='sigmoid'))
opt = SGD(learning_rate = 0, momentum = 0.1)

In [21]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [22]:
modelo = model.fit(X_train_token, y_train, batch_size=25, epochs=50, validation_data=(X_test_token, y_test))

Epoch 1/20
266/266 [==============================] - 578s 2s/step - loss: 1.7389 - accuracy: 0.3547 - val_loss: 1.5344 - val_accuracy: 0.4226
Epoch 2/20
266/266 [==============================] - 517s 2s/step - loss: 1.3775 - accuracy: 0.4981 - val_loss: 1.3878 - val_accuracy: 0.4857
Epoch 3/20
266/266 [==============================] - 465s 2s/step - loss: 1.2214 - accuracy: 0.5587 - val_loss: 1.3204 - val_accuracy: 0.5263
Epoch 4/20
266/266 [==============================] - 503s 2s/step - loss: 1.1235 - accuracy: 0.5956 - val_loss: 1.3180 - val_accuracy: 0.5263
Epoch 5/20
266/266 [==============================] - 526s 2s/step - loss: 1.0310 - accuracy: 0.6340 - val_loss: 1.3429 - val_accuracy: 0.5278
Epoch 6/20
266/266 [==============================] - 519s 2s/step - loss: 0.9568 - accuracy: 0.6604 - val_loss: 1.2798 - val_accuracy: 0.5534
Epoch 7/20
266/266 [==============================] - 490s 2s/step - loss: 0.8866 - accuracy: 0.6841 - val_loss: 1.2796 - val_accuracy: 0.5504

In [23]:
score, acc = model.evaluate(X_test_token, y_test,
                            batch_size=10)
print('Test score:', score)
print('Test accuracy:', acc)

67/67 [==============================] - 15s 178ms/step - loss: 1.6454 - accuracy: 0.5835
Test score: 1.645367980003357
Test accuracy: 0.583458662033081
